In [1]:
!git clone https://github.com/arzoovaswani415/Crowd_density_estimation.git

Cloning into 'Crowd_density_estimation'...
remote: Enumerating objects: 1556, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 1556 (delta 28), reused 52 (delta 14), pack-reused 1487 (from 1)
Receiving objects: 100% (1556/1556), 766.92 MiB | 34.35 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mv "/content/drive/MyDrive/Colab Notebooks/05_CSRNet.ipynb" \
    "/content/Crowd_density_estimation/notebooks/"


mv: cannot stat '/content/drive/MyDrive/Colab Notebooks/05_CSRNet.ipynb': No such file or directory


In [ ]:
import os, sys
os.chdir("/content/Crowd_density_estimation")
sys.path.append(os.getcwd())


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from src.dataset import ShanghaiTechDataset
from src.csrnet import CSRNet


In [ ]:
# Train data paths
train_img_dir = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/train_data/images"
train_gt_dir  = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/train_data/ground-truth-h5"

# Test data paths
test_img_dir = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/test_data/images"
test_gt_dir  = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/test_data/ground-truth-h5"


In [ ]:
train_dataset = ShanghaiTechDataset(
    train_img_dir,
    train_gt_dir,
    resize=(256, 256)
)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
model = CSRNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # lower LR

epochs = 10


In [ ]:
import torch.nn.functional as F

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for img, gt in train_loader:
        img = img.to(device)
        gt = gt.to(device)

        pred = model(img)

        # 🔥 CRITICAL: upsample to GT size
        pred = F.interpolate(
            pred,
            size=gt.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        loss = criterion(pred, gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {epoch_loss/len(train_loader):.4f}")


Epoch [1/10] | Loss: 0.0009
Epoch [2/10] | Loss: 0.0007
Epoch [3/10] | Loss: 0.0007
Epoch [4/10] | Loss: 0.0007
Epoch [5/10] | Loss: 0.0007
Epoch [6/10] | Loss: 0.0007
Epoch [7/10] | Loss: 0.0007
Epoch [8/10] | Loss: 0.0007
Epoch [9/10] | Loss: 0.0007
Epoch [10/10] | Loss: 0.0006


In [ ]:
train_img_dir = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/train_data/images"
train_gt_dir  = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/train_data/ground-truth-h5"

test_img_dir = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/test_data/images"
test_gt_dir  = "/content/drive/MyDrive/datasets/ShanghaiTech/part_A/test_data/ground-truth-h5"


In [ ]:
from src.dataset import ShanghaiTechDataset

train_dataset = ShanghaiTechDataset(
    train_img_dir,
    train_gt_dir,
    resize=(256, 256)
)

test_dataset = ShanghaiTechDataset(
    test_img_dir,
    test_gt_dir,
    resize=(256, 256)
)


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False
)


In [ ]:
import torch.nn.functional as F
import numpy as np

model.eval()
mae, mse = 0.0, 0.0

with torch.no_grad():
    for img, gt in test_loader:
        img = img.to(device)
        gt = gt.to(device)

        pred = model(img)

        # 🔥 MUST upsample here also
        pred = F.interpolate(
            pred,
            size=gt.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        gt_count   = gt.sum().item()
        pred_count = pred.sum().item()

        mae += abs(gt_count - pred_count)
        mse += (gt_count - pred_count) ** 2

mae /= len(test_loader)
rmse = np.sqrt(mse / len(test_loader))

print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")


MAE  : 218.88
RMSE : 351.69


In [ ]:
print("GT sum:", gt.sum().item())
print("Pred sum:", pred.sum().item())
print("GT shape:", gt.shape)
print("Pred shape:", pred.shape)



GT sum: 240.4397430419922
Pred sum: 272.19091796875
GT shape: torch.Size([1, 1, 256, 256])
Pred shape: torch.Size([1, 1, 256, 256])


In [ ]:
errors = []

with torch.no_grad():
    for img, gt in test_loader:
        img = img.to(device)
        gt = gt.to(device)

        pred = model(img)
        pred = F.interpolate(pred, size=gt.shape[-2:], mode="bilinear", align_corners=False)

        gt_c = gt.sum().item()
        pr_c = pred.sum().item()

        errors.append(abs(gt_c - pr_c))

errors = np.array(errors)

print("Min error :", errors.min())
print("Max error :", errors.max())
print("Mean error:", errors.mean())
print("Median error:", np.median(errors))


Min error : 0.356231689453125
Max error : 1805.6768493652344
Mean error: 218.87675040108817
Median error: 120.17417907714844


In [ ]:
import os
os.makedirs("results/csrnet", exist_ok=True)


In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

model.eval()
saved = 0
max_images = 5   # you can increase later

with torch.no_grad():
    for idx, (img, gt) in enumerate(test_loader):
        if saved >= max_images:
            break

        img = img.to(device)
        gt = gt.to(device)

        pred = model(img)
        pred = F.interpolate(
            pred,
            size=gt.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        gt_count = gt.sum().item()
        pred_count = pred.sum().item()

        plt.figure(figsize=(12,4))

        plt.subplot(1,3,1)
        plt.imshow(img[0].permute(1,2,0).cpu())
        plt.title("Input Image")
        plt.axis("off")

        plt.subplot(1,3,2)
        plt.imshow(gt[0,0].cpu(), cmap="jet")
        plt.title(f"GT Density\nCount={gt_count:.1f}")
        plt.axis("off")

        plt.subplot(1,3,3)
        plt.imshow(pred[0,0].cpu(), cmap="jet")
        plt.title(f"CSRNet Prediction\nCount={pred_count:.1f}")
        plt.axis("off")

        plt.tight_layout()
        plt.savefig(f"results/csrnet/sample_{idx}.png")
        plt.close()

        saved += 1
